In [2]:
def segmentation(paths):
    patient_no = ['100','102','103','104','105','106','107','108','109','111','112','113','114',
                '115','116','117','118','119','121','122','123','124','200','201','202','203','205',
                '207','208','209','210','212','213','214','215','217','219','220','221','222','223','228',
                '230','231','232','233','234'] 
    labels = []; leadML2_beats = []; leadV1_beats = []
    for k in range(0,len(patient_no)):
        ann = wfdb.rdann(paths[k][:-4], 'atr') 
        
        if wfdb.rdsamp(paths[k][:-4])[1].get('sig_name') == ['MLII', 'V1']:
            leadML2 = wfdb.rdsamp(paths[k][:-4])[0][:, 0]
            lead2 = wfdb.rdsamp(paths[k][:-4])[0][:, 1]
            peaks = ann.sample
            for j in range(len(peaks)-1):
                if peaks[j]>=180 and peaks[j]< len(leadML2)-180:
                    leadML2_beat = leadML2[peaks[j]-180:peaks[j]+180]
                    leadML2_beats.append(leadML2_beat)

                    leadV1_beat = lead2[peaks[j]-180:peaks[j]+180]
                    leadV1_beats.append(leadV1_beat)
                    labels.append(ann.symbol[j])
    return  leadML2_beats, leadV1_beats, labels   



'''We exclude specific annotations/beats; for more details about annotations go to: 
https://archive.physionet.org/physiobank/annotations.shtml#aux''' 
def exclude(labels,leadML2):
    '''We exclude specific annotations/beats(the ones from 'exclude_char'); 
    for more details about annotations go to: https://archive.physionet.org/physiobank/annotations.shtml#aux 
    Cathegories recommended by the AAMI standards:
        Normal = ['N','L','R','e','j']
        SVEB = ['A','a','J','S']
        VEB = ['V','E']
        F = ['F']
        Q = ['/', 'Q', 'f']
    '''
    exclude_char = ['~','|','!','[',']','"','no','+','x']
    zero_list=[]
    for i in exclude_char:
        for j in range(len(labels)):
            if labels[j]==i:
                labels[j] = 0  # Finds char from 'exclude_char' and replaces them with zero
                zero_list.append(j)  #find the indexes where labels=0 
    leadML2 = [leadML2[i] for i in range(0, len(labels)) if i not in zero_list] 
    labels = [labels[i] for i in range(0, len(labels)) if i not in zero_list] 
    
    
#     Normal = []; SVEB = []; VEB = []; F = []; Q = []
#     for j in range(len(labels)):
#         if labels[j]=='N' or labels[j]=='L' or labels[j]=='R' or labels[j]=='e' or labels[j]=='j':
#             labels[j] = 0  # N
#             Normal.append(leadML2[j])
#         elif labels[j]=='A' or labels[j]=='a' or labels[j]=='J' or labels[j]=='S':
#             labels[j] = 1  #'SVEB'
#             SVEB.append(leadML2[j])
#         elif labels[j]=='V' or labels[j]=='E':
#             labels[j] = 2  # 'VEB'
#             VEB.append(leadML2[j])
#         elif labels[j]=='F':
#             labels[j] = 3  #'F'
#             F.append(leadML2[j])
#         elif labels[j]=='/' or labels[j]=='Q' or labels[j]=='f':
#             labels[j] = 4  #'Q'
#             Q.append(leadML2[j])
    Normal = []; AA = []; LBBB = []; RBBB = []; 
    for j in range(len(labels)):
        if labels[j]=='N':
            labels[j] = 0  # N
            Normal.append(leadML2[j])
        elif labels[j]=='A':
            labels[j] = 1  #'SVEB'
            AA.append(leadML2[j])
        elif labels[j]=='L':
            labels[j] = 2  # 'VEB'
            LBBB.append(leadML2[j])

        elif labels[j]=='R':
            labels[j] = 3  #'F'
            RBBB.append(leadML2[j])

            
    return labels,Normal, AA, LBBB, RBBB


In [ ]:
## DENOISE THE ECG SIGNALS   
import scipy
from scipy.signal import savgol_filter
def Savitzky_Golay(signals,fs):
    signals_filtered = []
   
    
    for i in range(0,len(signals)):
        # Savitzky-Golay filter
        y_filtered = savgol_filter(signals[i], 15, 2)
        signals_filtered.append(y_filtered) 
    return  signals_filtered
# Normalize signals between -1 and 1 
def normalise(filtered_signals,a,b):
    normalized_signals = []
    for i in range (0,len(filtered_signals)):
        signal = filtered_signals[i]
        #print(i)
        A = np.min(signal)
        B = np.max(signal)
        C = (a-b)/(A-B)
        k = (C*A - a)/C
        normalized = (signal-k)*C

        normalized_signals.append(normalized)
    return normalized_signals

In [ ]:
def down_sample(baseline_segments): 
    from scipy import signal
    for i in range(0,len(baseline_segments)):
        baseline_segments[i] = signal.decimate(baseline_segments[i], 4)  # down sample signals to 250
    return baseline_segments  